In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, pi
import pandas as pd
import os
import random
from scipy import fftpack
import pandas as pd
import math

os.environ['OPENBLAS_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'

In [2]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{pmatrix}']
    rv += ['  ' + ' & '.join(lines[l].split()) + r'\\' for l in range(2,len(lines))]
    rv +=  [r'\end{pmatrix}']
    print('\n'.join(rv))
    return

In [3]:
import scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega" "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = np.array(tt)
    yy = np.array(yy)
    ff = np.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(np.fft.fft(yy))
    guess_freq = abs(ff[np.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = np.std(yy) * 2.**0.5
    guess_offset = np.mean(yy)
    guess = np.array([guess_amp, 2.*np.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * np.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*np.pi)
    fitfunc = lambda t: A * np.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": np.max(pcov), "rawres": (guess,popt,pcov)}

In [4]:
def gen_ladder_operators(n, N):
    """
    Used by gen_free_terms(n, N)
    Generate atomic and photon ladder operators in terms of their product states.
    n = number of atoms
    N = number of Folk states

    return
    ------
    list of lowering operators that is n+1 length
    last term is cavity lowering operator
    """
    a_atom = destroy(2) # Ladder operator atom
    id_atom = qeye(2) # Identity atom
    a_cavity = destroy(N)
    id_cavity = qeye(N)
    terms = []

    # Atomic ladder operators
    for i in range(0,n):
        term = [id_atom] * (n)
        term[i] = a_atom
        term.append(id_cavity)
        terms.append(tensor(term))
    
    # Cavity ladder operator
    term = [id_atom] * (n)
    term.append(a_cavity)
    terms.append(tensor(term))

    return terms

def gen_free_terms(n, N, w0, wc):
    """
    Generate free terms in quantum optics system
    n = number of atoms
    N = number of Folk states
    """
    # Generate list of ladder operators
    ladders = gen_ladder_operators(n, N)

    # Generate Hamiltonian of free terms
    H = 0
    for i in range(0,n):
        free_term = w0 * ladders[i].dag() * ladders[i]
        H += free_term
    H += wc * ladders[n].dag() * ladders[n]
    return H

def gen_interaction_terms(n, N, c = []):
    """
    Generate interaction terms (No RWA)
    n = number of atoms
    N = number of Folk states
    c = list of coupling weights
    """
    # check for coupling list
    if len(c) == 0:
        c = [1]*n

    
    # Generate list of ladder operators
    ladders = gen_ladder_operators(n, N)
    
    H = 0 # Initialise Hamiltonian
    
    for i in range(0, n):
        term = c[i] * (ladders[n].dag() * ladders[i] + ladders[n] * ladders[i].dag())
        H += term
   
    return H

def gen_psi(states, N):
    """
    Generate starting psi0 state. Takes input of states as list:
    
    N = Folk state

    Example
    -------
    [0,0,1,0,3]

    Would relate to three gound state atoms, one excited atom and 
    three photons in the cavity.
    """
    psi = [] # initiate psi
    for i in states[0: len(states)-1]:
        c1 = i
        c2 = 1 - i
        state = c1 * basis(2,1) + c2 * basis(2,0)
        psi.append(state)
    psi.append(basis(N, states[-1]))
    return tensor(psi).unit()


def gen_exp(n, N):
    """
    Generate expectation values for populations of atomic and cavity
    excitations
    n = number of atoms
    N = number of photons

    return
    ------
    Returns list of length two. Where first entry is expectation value 
    of atom excitation number. Secound entry is photon number.
    """

    # Generate list of ladder operators
    ladders = gen_ladder_operators(n, N)
    
    expAtom = 0
    expPhoton = 0

    for i in range(0,n):
        expAtom += ladders[i].dag() * ladders[i]
    
    expPhoton = ladders[n].dag()*ladders[n]

    return [expAtom, expPhoton]

def gen_h(n, N, c, w0, wc, g):
    """
    Generate disorded TCM Hamiltonian of n atoms
    n = number of atoms
    N = number of Folk states
    c = array of coupling weights
    """
    H0 = gen_free_terms(n, N, w0, wc)
    H1 = gen_interaction_terms(n, N, c)
    return H0 + g * H1

def sim(H, psi0, times, n, N, kappa = 0):
    operators = gen_ladder_operators(n, N)
    cavityDecay = kappa * operators[-1]
    c_ops = [cavityDecay]

    exp = gen_exp(n, N)
    output = mesolve(H, psi0, times, c_ops, exp, options = Options(num_cpus=8))
    return output


In [5]:
def ftrans(df, val):
    """
    Perform FT
    Return array [freq, amplitude]
    """
    n_samples = len(df['time'])
    t0 = df['time'][0]
    t1 = df['time'][n_samples - 1]
    
    result = df[val]

    result_detrend = result - np.mean(result)
    np_fft = np.fft.fft(result_detrend)
    amplitudes = 4 / n_samples * np.abs(np_fft) 
    frequencies = np.fft.fftfreq(n_samples) * n_samples * 1 / (t1 - t0)

    return [frequencies[:len(frequencies) // 2], amplitudes[:len(np_fft) // 2]] 

def expo(x, c):
    return math.exp(- x * c)

def weighting(df_in, c):
    df = df_in.copy()
    for i in [x for x in list(df.columns) if isinstance(x, numbers.Number)]:
        df['et'] = df['time'].apply(expo, args=(c,))
        df[i] = df[i] - np.mean(df[i])
        df[i] = df[i] * df['et']
    return df

def diff_spec(x, w1, w2):
    x1 = np.gradient(x)
    x2 = np.gradient(x1)
    x3 = np.gradient(x2)
    x4 = np.gradient(x3)

    return x - w1 * x2 + w2 * x4

In [6]:
import numbers

def generator(coupling, p1=1, p2=1, p3=1, times= np.linspace(0, 5000*10**-9, 10000), kappa=0):
    # Run simulation
    out = sim_handler(coupling, p1, p2, p3, times, kappa)
    df = pd.DataFrame(out)

    # Prepare plots of atoms
    dfproc = pd.DataFrame()
    dfproc[0] = [sum(i) for i in zip(*df['data'])]
    if (df['n']==1).any() == True:
        dfproc[1] = [sum(i) for i in zip(*df[df['n'] == 1]['data'])]
    if (df['n']==2).any() == True:
        dfproc[2] = [sum(i) for i in zip(*df[df['n'] == 2]['data'])]
    if (df['n']==3).any() == True:   
        dfproc[3] = [sum(i) for i in zip(*df[df['n'] == 3]['data'])]
    dfproc['time'] = times
    return dfproc


def event_helper(df, n, w1, w2):
    f = ftrans(df, n)
    f_ds = diff_spec(f[1], w1, w2)
    return f_ds

def event(dfproc, weight, coupling, w1=1, w2=1, p1=1, p2=1, p3=1):
    #times = np.linspace(0, 4*5000*10**-9, 1000)
    
    # Apply weighting
    df_w = weighting(dfproc, weight)
    
    fig = plt.figure(figsize=(10,5))
    fig.add_subplot(1,2,1)

    f = ftrans(df_w, 0)
    f_ds = event_helper(df_w, 0, w1, w2)
    plt.plot(f[0], f_ds, label = 'all')

    col = ['tab:blue', 'tab:blue', 'tab:red', 'tab:green']

    for i in [x for x in list(df_w.columns) if isinstance(x, numbers.Number)]:  
        if i != 0:
            f_ds = event_helper(df_w, i, w1, w2)
            plt.plot(f[0], f_ds, label = str(i) + ' atoms', c = col[i], alpha = 0.3) 

    plt.xlim([0,1.5e7])
    plt.legend()
    
    fig.add_subplot(1,2, 2)
    plt.plot(df_w['time'], df_w[0], label='all')
    for i in [x for x in list(df_w.columns) if isinstance(x, numbers.Number)]:  
        if i != 0:
            plt.plot(df_w['time'], df_w[i], label = str(i) + ' atoms', c = col[i], alpha = 0.3)

    #plt.plot(df3['time'], df3[sig], label='3 Atoms')
    plt.legend()
    #wout.clear_output()
    plt.show()
    

In [9]:
import multiprocessing as mp

def coupling_study(sigma, p1, p2, p3, times, kappa, return_l):  
    # Distribution of coupling strengths
    mu = 1
    outputs = []

    w0 = 19.556*10**9 # energy splitting of atom
    wc = 19.556*10**9 # energy splitting of cavity
    N =  4 # Number of Folk states
    g = 3*10**6 * 2 * np.pi# Coupling Strenght

    for i in range(20):
        output = {}

        n = random.choices(population= [1,2,3], weights=[p1, p2, p3])[0]
        c = np.random.normal(mu, sigma, n) 

        # Generate hamiltonian
        H = gen_h(n, N, c, w0, wc, g)

        # Generate psi0
        mu_state = 0.0
        sigma_state = 0
        state = abs(np.random.normal(mu_state, sigma_state, n))

        state = [0]*n
        state.append(1)
        psi0 = gen_psi(state, N)

        #print(n)
        # Run simulations
        data = sim(H, psi0, times, n, N, kappa)

        output['data'] = data.expect[1]
        output['n'] = n
        outputs.append(output)
        

    return_l.append(outputs)

def sim_handler(sigma,p1,p2,p3, times, kappa):
    #coupling_study(sig,p1,p2,p3,times)
    manager = mp.Manager()
    return_l = manager.list()
    procs = []
    for i in range(mp.cpu_count()):
         proc = mp.Process(target=coupling_study, args=(sigma,p1,p2,p3,times, kappa, return_l))
         procs.append(proc)
         proc.start()
    
    for p in procs:
         p.join()
    
    outputs = return_l
    flatten = lambda l: [item for sublist in l for item in sublist]
    return flatten(outputs)

    #output = [sum(i) for i in zip(*flatten(outputs))]
    #plt.plot(times, output)

In [10]:
import ipywidgets as widgets
from IPython.display import display


global weight
global coupling
global p1
global p2
global p3
global w1
global w2
global df_int
global times
global multiT
global kappa

times = np.linspace(0, 1*5000*10**-9, 1000)
weight = 0
coupling = 0
p1 = 1
p2 = 0
p3 = 0
w1 = 0
w2 = 0
kappa = 0

#df_int = generator(coupling, p1, p2, p3, times)

coupling_slider = widgets.FloatSlider(min=0, max=1, step=0.025, value = coupling, description ='sig')
p1_slider = widgets.FloatSlider(min=0, max=3, step=0.1,value = p1, description ='p1')
p2_slider = widgets.FloatSlider(min=0, max=3, step=0.1, value = p2, description ='p2')
p3_slider = widgets.FloatSlider(min=0, max=3, step=0.1, value = p3, description ='p3')
weight_slider = widgets.FloatSlider(min=0, max=1e6, step=1e3,value = weight, description ='weight')
w1_slider = widgets.FloatSlider(min=0, max=5, step=0.05, value = w1, description ='w1')
w2_slider = widgets.FloatSlider(min=0, max=5, step=0.05, value = w2, description ='w2')
time_slider = widgets.FloatSlider(min=0, max= 10, step= 0.1, value=1, description='time')

kappa_slider = widgets.FloatSlider(min=0, max= 500, step= 10, value=0, description='kappa')

wout = widgets.Output()

display( coupling_slider, p1_slider, p2_slider, p3_slider, weight_slider, w1_slider, w2_slider, time_slider, kappa_slider, wout)

def handle_weight(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global weight
        weight = change['new']
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_coupling(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global coupling
        coupling = change['new']
        global df_int 
        df_int = generator(coupling, p1, p2, p3, times, kappa)
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_w1(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global w1
        w1 = change['new']
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_w2(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global w2
        w2 = change['new']
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_p1(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global p1
        p1 = change['new']
        global df_int 
        df_int = generator(coupling, p1, p2, p3, times, kappa)
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_p2(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global p2
        p2 = change['new']
        global df_int 
        df_int = generator(coupling, p1, p2, p3, times, kappa)
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_p3(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global p3
        p3 = change['new']
        global df_int 
        df_int = generator(coupling, p1, p2, p3, times, kappa)
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_time(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global multiT
        global times
        multiT = change['new']
        times = np.linspace(0, multiT*5000*10**-9, 1000)
        global df_int 
        df_int = generator(coupling, p1, p2, p3, times, kappa)
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

def handle_kappa(change):
    wout.clear_output(wait=True)
    #plt.clf()
    with wout:
        global kappa
        kappa = change['new']
        global df_int 
        df_int = generator(coupling, p1, p2, p3, times, kappa)
        event(df_int, weight, coupling, w1, w2, p1 ,p2 , p3)

#widgets.interact(event, weight = w_slider, coupling = sig_slider)

weight_slider.observe(handle_weight, names='value')
coupling_slider.observe(handle_coupling, names='value')
w1_slider.observe(handle_w1, names='value')
w2_slider.observe(handle_w2, names='value')
p1_slider.observe(handle_p1, names='value')
p2_slider.observe(handle_p2, names='value')
p3_slider.observe(handle_p3, names='value')
time_slider.observe(handle_time, names='value')
kappa_slider.observe(handle_kappa, names='value')

FloatSlider(value=0.0, description='sig', max=1.0, step=0.025)

FloatSlider(value=1.0, description='p1', max=3.0)

FloatSlider(value=0.0, description='p2', max=3.0)

FloatSlider(value=0.0, description='p3', max=3.0)

FloatSlider(value=0.0, description='weight', max=1000000.0, step=1000.0)

FloatSlider(value=0.0, description='w1', max=5.0, step=0.05)

FloatSlider(value=0.0, description='w2', max=5.0, step=0.05)

FloatSlider(value=1.0, description='time', max=10.0)

FloatSlider(value=0.0, description='kappa', max=500.0, step=10.0)

Output()